## Data Cleaning

#### Importing pandas

In [1]:
import pandas as pd

#### Importing the csv files for cleaning the data

In [2]:
bookings_data = pd.read_csv('datasets/fact_bookings.csv')
df_date = pd.read_csv('datasets/dim_date.csv')
df_rooms = pd.read_csv('datasets/dim_rooms.csv')
df_fact_agg_bookings = pd.read_csv('datasets/fact_aggregated_bookings.csv')
df_hotels = pd.read_csv('datasets/dim_hotels.csv')

#### Let's explore bookings data

In [3]:
bookings_data.describe()

,property_id,no_guests,ratings_given,revenue_generated,revenue_realized
count,134590.000000,134587.000000,56683.000000,1.345900e+05,134590.000000
mean,18061.113493,2.036170,3.619004,1.537805e+04,12696.123256
std,1093.055847,1.034885,1.235009,9.303604e+04,6928.108124
min,16558.000000,-17.000000,1.000000,6.500000e+03,2600.000000
25%,17558.000000,1.000000,3.000000,9.900000e+03,7600.000000
50%,17564.000000,2.000000,4.000000,1.350000e+04,11700.000000
75%,18563.000000,2.000000,5.000000,1.800000e+04,15300.000000
max,19563.000000,6.000000,5.000000,2.856000e+07,45220.000000


-Here we can observe **negative values** in no_guests column.  
-Having a **negative value** for no_guests is an error

Now lets check the no of negative records for no_guests column

In [4]:
bookings_data[bookings_data.no_guests < 0]

,booking_id,property_id,booking_date,check_in_date,checkout_date,no_guests,room_category,booking_platform,ratings_given,booking_status,revenue_generated,revenue_realized
0,May012216558RT11,16558,27-04-22,1/5/2022,2/5/2022,-3.0,RT1,direct online,1.0,Checked Out,10010,10010
3,May012216558RT14,16558,28-04-22,1/5/2022,2/5/2022,-2.0,RT1,others,NaN,Cancelled,9100,3640
17924,May122218559RT44,18559,12/5/2022,12/5/2022,14-05-22,-10.0,RT4,direct online,NaN,No Show,20900,20900
18020,May122218561RT22,18561,8/5/2022,12/5/2022,14-05-22,-12.0,RT2,makeyourtrip,NaN,Cancelled,9000,3600
18119,May122218562RT311,18562,5/5/2022,12/5/2022,17-05-22,-6.0,RT3,direct offline,5.0,Checked Out,16800,16800
18121,May122218562RT313,18562,10/5/2022,12/5/2022,17-05-22,-4.0,RT3,direct online,NaN,Cancelled,14400,5760
56715,Jun082218562RT12,18562,5/6/2022,8/6/2022,13-06-22,-17.0,RT1,others,NaN,Checked Out,6500,6500
119765,Jul202219560RT220,19560,19-07-22,20-07-22,22-07-22,-1.0,RT2,others,NaN,Checked Out,13500,13500
134586,Jul312217564RT47,17564,30-07-22,31-07-22,1/8/2022,-4.0,RT4,logtrip,2.0,Checked Out,38760,38760


Let's check the size of the data before removing the records with **negative values**

In [5]:
bookings_data.shape

(134590, 12)

Let's remove the **negative values** for no_guests

In [6]:
bookings_data = bookings_data[bookings_data.no_guests > 0]
bookings_data.shape

(134578, 12)

We can observe that 12 records have been removed

#### Now let's explore the revenue_generated column

In [7]:
bookings_data.describe()

,property_id,no_guests,ratings_given,revenue_generated,revenue_realized
count,134578.000000,134578.000000,56679.000000,1.345780e+05,134578.000000
mean,18061.143315,2.036744,3.619048,1.537804e+04,12696.011822
std,1093.053454,1.031710,1.234970,9.304015e+04,6927.841641
min,16558.000000,1.000000,1.000000,6.500000e+03,2600.000000
25%,17558.000000,1.000000,3.000000,9.900000e+03,7600.000000
50%,17564.000000,2.000000,4.000000,1.350000e+04,11700.000000
75%,18563.000000,2.000000,5.000000,1.800000e+04,15300.000000
max,19563.000000,6.000000,5.000000,2.856000e+07,45220.000000


In [12]:
bookings_data.revenue_generated.min().item(),bookings_data.revenue_generated.max().item() 

(6500, 28560000)

Here we can clearly observe that the maximum revenue generated is 28.5million which is extemely high for 1 day

Let's identify the outliers 

We will use the below formula to find the outliers:
- Outliers = any value > mean + 3 * std_dev
- mean = represents the avg_value of all the records
- std_dev = represents the value above/below its mean

#### Computing the mean, std_dev

In [13]:
mean, std_dev = bookings_data.revenue_generated.mean(), bookings_data.revenue_generated.std()

In [16]:
mean, std_dev = mean.item(), std_dev.item()
mean, std_dev

(15378.036937686695, 93040.1549314641)

#### Setting the upper and lower thresold for outliers

In [17]:
higher_limit = mean + 3*std_dev
higher_limit

294498.50173207896

In [18]:
lower_limit = mean - 3*std_dev
lower_limit

-263742.4278567056

#### Let's check the no of records which are below the lower thresold

In [19]:
bookings_data[bookings_data.revenue_generated < lower_limit]

,booking_id,property_id,booking_date,check_in_date,checkout_date,no_guests,room_category,booking_platform,ratings_given,booking_status,revenue_generated,revenue_realized


#### Let's check the no of records which are below the upper thresold

In [21]:
bookings_data[bookings_data.revenue_generated > higher_limit]

,booking_id,property_id,booking_date,check_in_date,checkout_date,no_guests,room_category,booking_platform,ratings_given,booking_status,revenue_generated,revenue_realized
2,May012216558RT13,16558,28-04-22,1/5/2022,4/5/2022,2.0,RT1,logtrip,5.0,Checked Out,9100000,9100
111,May012216559RT32,16559,29-04-22,1/5/2022,2/5/2022,6.0,RT3,direct online,NaN,Checked Out,28560000,28560
315,May012216562RT22,16562,28-04-22,1/5/2022,4/5/2022,2.0,RT2,direct offline,3.0,Checked Out,12600000,12600
562,May012217559RT118,17559,26-04-22,1/5/2022,2/5/2022,2.0,RT1,others,NaN,Cancelled,2000000,4420
129176,Jul282216562RT26,16562,21-07-22,28-07-22,29-07-22,2.0,RT2,direct online,3.0,Checked Out,10000000,12600


#### Removing the records where revenue generated is more than upper thresold

Size before removing

In [23]:
bookings_data.shape

(134578, 12)

Size after removing

In [24]:
bookings_data = bookings_data[bookings_data.revenue_generated < higher_limit]
bookings_data.shape

(134573, 12)

#### Let's check for null values

In [26]:
df_fact_agg_bookings.isnull().sum()

property_id            0
check_in_date          0
room_category          0
successful_bookings    0
capacity               2
dtype: int64

Here we can observe that capacity column has 2 null values

In [39]:
median_capacity = df_fact_agg_bookings.capacity.median().item()
median_capacity

25.0

In [38]:
df_fact_agg_bookings[df_fact_agg_bookings.capacity.isna()]

,property_id,check_in_date,room_category,successful_bookings,capacity
8,17561,1-May-22,RT1,22,NaN
14,17562,1-May-22,RT1,12,NaN


#### Filling two null values with median value of the same column 

In [43]:
df_fact_agg_bookings.fillna(median_capacity, inplace=True)

In [45]:
df_fact_agg_bookings.isnull().sum()

property_id            0
check_in_date          0
room_category          0
successful_bookings    0
capacity               0
dtype: int64

#### Now let's check for successful bookings > capacity and if any let's remove those data points 

In [ ]:
df_fact_agg_bookings[df_fact_agg_bookings.successful_bookings > df_fact_agg_bookings.capacity]


,property_id,check_in_date,room_category,successful_bookings,capacity
3,17558,1-May-22,RT1,30,19.0
12,16563,1-May-22,RT1,100,41.0
4136,19558,11-Jun-22,RT2,50,39.0
6209,19560,2-Jul-22,RT1,123,26.0
8522,19559,25-Jul-22,RT1,35,24.0
9194,18563,31-Jul-22,RT4,20,18.0


In [47]:
df_fact_agg_bookings.shape

(9200, 5)

#### Remove these data points from the data frame

In [48]:
df_fact_agg_bookings =  df_fact_agg_bookings[df_fact_agg_bookings.successful_bookings <= df_fact_agg_bookings.capacity]

df_fact_agg_bookings.shape

(9194, 5)